In [1]:
import pandas as pd
pd.set_option("display.max_columns",None)
import warnings
warnings.filterwarnings("ignore")
import json
import requests
from geopy.distance import geodesic


In [3]:
df1 = pd.read_csv("F:/Desktop/DTM19/Projects/Singapore/ResaleFlatPricesBasedonApprovalDate2000Feb2012.csv")
df2 = pd.read_csv("F:/Desktop/DTM19/Projects/Singapore/ResaleFlatPricesBasedonApprovalDate19901999.csv")
df3 = pd.read_csv("F:/Desktop/DTM19/Projects/Singapore/ResaleFlatPricesBasedonRegistrationDateFromJan2015toDec2016.csv")
df4 = pd.read_csv("F:/Desktop/DTM19/Projects/Singapore/ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv")
df5 = pd.read_csv("F:/Desktop/DTM19/Projects/Singapore/ResaleFlatPricesBasedonRegistrationDateFromMar2012toDec2014.csv")

In [4]:
df = pd.concat([df1,df2,df3,df4,df5], axis = 0)

In [5]:
df.shape
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 931709 entries, 0 to 52202
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                931709 non-null  object 
 1   town                 931709 non-null  object 
 2   flat_type            931709 non-null  object 
 3   block                931709 non-null  object 
 4   street_name          931709 non-null  object 
 5   storey_range         931709 non-null  object 
 6   floor_area_sqm       931709 non-null  float64
 7   flat_model           931709 non-null  object 
 8   lease_commence_date  931709 non-null  int64  
 9   resale_price         931709 non-null  float64
 10  remaining_lease      222659 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 85.3+ MB


Null Values

In [6]:
df.isnull().mean()*100

month                   0.000000
town                    0.000000
flat_type               0.000000
block                   0.000000
street_name             0.000000
storey_range            0.000000
floor_area_sqm          0.000000
flat_model              0.000000
lease_commence_date     0.000000
resale_price            0.000000
remaining_lease        76.102088
dtype: float64

In [7]:
df = df.dropna()
df.isnull().sum()

month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
resale_price           0
remaining_lease        0
dtype: int64

In [8]:
len(df['street_name'].unique())
#len(df['block'].unique())
df['floor_area_sqm'].min()
df['lease_commence_date'].min()

np.int64(1966)

In [9]:
list_of_mrt = [
    'Jurong East MRT Station',
    'Bukit Batok MRT Station',
    'Bukit Gombak MRT Station',
    'Choa Chu Kang MRT Station',
    'Yew Tee MRT Station',
    'Kranji MRT Station',
    'Marsiling MRT Station',
    'Woodlands MRT Station',
    'Admiralty MRT Station',
    'Sembawang MRT Station',
    'Canberra MRT Station',
    'Yishun MRT Station',
    'Khatib MRT Station',
    'Yio Chu Kang MRT Station',
    'Ang Mo Kio MRT Station',
    'Bishan MRT Station',
    'Braddell MRT Station',
    'Toa Payoh MRT Station',
    'Novena MRT Station',
    'Newton MRT Station',
    'Orchard MRT Station',
    'Somerset MRT Station',
    'Dhoby Ghaut MRT Station',
    'City Hall MRT Station',
    'Raffles Place MRT Station',
    'Marina Bay MRT Station',
    'Marina South Pier MRT Station',
    'Pasir Ris MRT Station',
    'Tampines MRT Station',
    'Simei MRT Station',
    'Tanah Merah MRT Station',
    'Bedok MRT Station',
    'Kembangan MRT Station',
    'Eunos MRT Station',
    'Paya Lebar MRT Station',
    'Aljunied MRT Station',
    'Kallang MRT Station',
    'Lavender MRT Station',
    'Bugis MRT Station',
    'Tanjong Pagar MRT Station',
    'Outram Park MRT Station',
    'Tiong Bahru MRT Station',
    'Redhill MRT Station',
    'Queenstown MRT Station',
    'Commonwealth MRT Station',
    'Buona Vista MRT Station',
    'Dover MRT Station',
    'Clementi MRT Station',
    'Chinese Garden MRT Station',
    'Lakeside MRT Station',
    'Boon Lay MRT Station',
    'Pioneer MRT Station',
    'Joo Koon MRT Station',
    'Gul Circle MRT Station',
    'Tuas Crescent MRT Station',
    'Tuas West Road MRT Station',
    'Tuas Link MRT Station',
    'Expo MRT Station',
    'Changi Airport MRT Station',
    'HarbourFront MRT Station',
    'Chinatown MRT Station',
    'Clarke Quay MRT Station',
    'Little India MRT Station',
    'Farrer Park MRT Station',
    'Boon Keng MRT Station',
    'Potong Pasir MRT Station',
    'Woodleigh MRT Station',
    'Serangoon MRT Station',
    'Kovan MRT Station',
    'Hougang MRT Station',
    'Buangkok MRT Station',
    'Sengkang MRT Station',
    'Punggol MRT Station',
    'Bras Basah MRT Station',
    'Esplanade MRT Station',
    'Promenade MRT Station',
    'Nicoll Highway MRT Station',
    'Stadium MRT Station',
    'Mountbatten MRT Station',
    'Dakota MRT Station',
    'MacPherson MRT Station',
    'Tai Seng MRT Station',
    'Bartley MRT Station',
    'Lorong Chuan MRT Station',
    'Marymount MRT Station',
    'Caldecott MRT Station',
    'Botanic Gardens MRT Station',
    'Farrer Road MRT Station',
    'Holland Village MRT Station',
    'one-north MRT Station',
    'Kent Ridge MRT Station',
    'Haw Par Villa MRT Station',
    'Pasir Panjang MRT Station',
    'Labrador Park MRT Station',
    'Telok Blangah MRT Station',
    'Bayfront MRT Station',
    'Bukit Panjang MRT Station',
    'Cashew MRT Station',
    'Hillview MRT Station',
    'Beauty World MRT Station',
    'King Albert Park MRT Station',
    'Sixth Avenue MRT Station',
    'Tan Kah Kee MRT Station',
    'Stevens MRT Station',
    'Rochor MRT Station',
    'Downtown MRT Station',
    'Telok Ayer MRT Station',
    'Fort Canning MRT Station',
    'Bencoolen MRT Station',
    'Jalan Besar MRT Station',
    'Bendemeer MRT Station',
    'Geylang Bahru MRT Station',
    'Mattar MRT Station',
    'Ubi MRT Station',
    'Kaki Bukit MRT Station',
    'Bedok North MRT Station',
    'Bedok Reservoir MRT Station',
    'Tampines West MRT Station',
    'Tampines East MRT Station',
    'Upper Changi MRT Station',
    'Woodlands North MRT Station',
    'Woodlands South MRT Station'
]


In [10]:
mrt_latt = []
mrt_long = []

for i in range(len(list_of_mrt)):
    mrt_station = list_of_mrt[i]
    url = "https://www.onemap.gov.sg/api/common/elastic/search?searchVal=" + str(mrt_station)+"&returnGeom=Y&getAddrDetails=Y&pageNum=1"
    response = requests.request("GET", url)
    data_mrt=json.loads(response.content)

    if data_mrt['found'] != 0:
        mrt_latt.append(data_mrt["results"][0]["LATITUDE"])
        mrt_long.append(data_mrt["results"][0]["LONGITUDE"])

        print (str(mrt_station)+",Lat: "+data_mrt['results'][0]['LATITUDE'] +" Long: "+data_mrt['results'][0]['LONGITUDE'])

    else:
        mrt_lat.append('NotFound')
        mrt_lat.append('NotFound')
        print ("No Results")

Jurong East MRT Station,Lat: 1.33329506563598 Long: 103.742153884191
Bukit Batok MRT Station,Lat: 1.34903331201636 Long: 103.749566478309
Bukit Gombak MRT Station,Lat: 1.35861159094192 Long: 103.751790910733
Choa Chu Kang MRT Station,Lat: 1.38536316540225 Long: 103.744370779756
Yew Tee MRT Station,Lat: 1.39729771118639 Long: 103.74735767326
Kranji MRT Station,Lat: 1.4250690629411 Long: 103.762099449562
Marsiling MRT Station,Lat: 1.43262603485889 Long: 103.774051197031
Woodlands MRT Station,Lat: 1.43681962961519 Long: 103.786066799253
Admiralty MRT Station,Lat: 1.44058856161847 Long: 103.800990519771
Sembawang MRT Station,Lat: 1.44902668521092 Long: 103.820139486468
Canberra MRT Station,Lat: 1.44307664075699 Long: 103.829702590959
Yishun MRT Station,Lat: 1.42952522867762 Long: 103.834995141847
Khatib MRT Station,Lat: 1.41758906054002 Long: 103.833061043125
Yio Chu Kang MRT Station,Lat: 1.38149882492597 Long: 103.845170704861
Ang Mo Kio MRT Station,Lat: 1.36993284962264 Long: 103.8495580